In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
import os
from sklearn.metrics import mean_squared_error

In [2]:
#clean the vertical white space
def clear_white_column(img,max_changes,restricted_pixel):
    gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    delete=[]
    for i in restricted_pixel:
        a = np.bincount(gray[:,i])
        if (sum(a == 0) > max_changes):
            delete.append(i)
    for j in delete[::-1]:
        img=np.delete(img,j,1)
    return img

In [3]:
#clear horizantal white space 
def clear_white_edge(img):
    a=img.shape[0]
    k=10
    prob_index_0=[i for i in range(0,k+1)]
    prob_index_3=[i for i in range(a-k,a)]
    gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    upper=[]
    lower=[]
    for i in prob_index_0:
        upper.append(sum(gray[i]>220))
    for i in prob_index_3:
        lower.append(sum(gray[i]>220))
    return len(upper)-upper[::-1].index(img.shape[1]),len(lower)-lower.index(img.shape[1])
    

In [4]:
#clear  black area horizontaly
def clear_black_edge(img):
    a=img.shape[0]
    k=20
    prob_index_0=[i for i in range(0,k+1)]
    prob_index_3=[i for i in range(a-k,a)]
    gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    upper=[]
    lower=[]
    for i in prob_index_0:
        upper.append(sum(gray[i]<35))
    for i in prob_index_3:
        lower.append(sum(gray[i]<35))
    return len(upper)-upper[::-1].index(max(upper)),len(lower)-lower.index(max(lower))
    

In [5]:
# show the data image
def show(data):
    for i in data:
        cv.imshow("img",i)
        cv.waitKey(0)
        cv.destroyAllWindows()

In [6]:
#take image and return two black point for middle of image
def find_black_lines_middle(img):
    a=img.shape[0]//3
    b=2*a
    k=20
    prob_index1=[i for i in range(a-k,a+k+1)]
    prob_index2=[i for i in range(b-k,b+k+1)]
    gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    upper=[]
    lower=[]
    for i in prob_index1:
        upper.append(sum(gray[i]<40))
    for i in prob_index2:
        lower.append(sum(gray[i]<40))
    return a+upper.index(max(upper))-k,b+lower.index(max(lower))-k

In [7]:
img_name=os.listdir("data")

In [8]:
# read image and preprocess
data=[]
k=15
for i in img_name:
    img=cv.imread("data/"+i)
    row=[i for i in range(k)]
    column=[i for i in range(k)]
    column1=[i for i in range(img.shape[1]-k,img.shape[1])]
    row1=[i for i in range(img.shape[0]-k,img.shape[0])]
    row.extend(row1)
    column.extend(column1)
    img1=clear_white_column(img,70,column)
    upper,lower=clear_white_edge(img1)
    img1=img1[upper:img1.shape[0]-lower]
    upper,lower=clear_black_edge(img1)
    img1=img1[upper:img1.shape[0]-lower,]
    data.append(img1)

In [9]:
# save image in folder
def save_img(data,colorspace):
    for i in range(len(data)):
        cv.imwrite("Edited"+str(i)+colorspace+".jpg",data[i])
    print("Images saved for "+colorspace+" space")
    o

In [ ]:
os.chdir("edit")

In [10]:
R=[]
G=[]
B=[]
for i in data:
    upper,lower=find_black_lines_middle(i)
    B.append(i[:upper,:,0])
    G.append(i[upper+2:lower,:,1])
    R.append(i[lower+2:,:,2])

In [12]:
save_img(R,"R")
save_img(G,"G")
save_img(B,"B")

Images saved for R space
Images saved for G space
Images saved for B space


In [13]:
os.chdir("..")

In [14]:
def build_filters():
    filters = []
    ksize = 30
    for theta in np.arange(0, np.pi, np.pi / 16):
        kern = cv.getGaborKernel((ksize, ksize), 4.0, theta, 10.0, 0.5, 0.8, ktype=cv.CV_32F)
        kern /= 1.5 * kern.sum()
        filters.append(kern)
    return filters
def process(img, filters):
    accum = np.zeros_like(img)
    for kern in filters:
        fimg = cv.filter2D(img, cv.CV_8UC3, kern)
        np.maximum(accum, fimg, accum)
    return accum

In [15]:
def equalization_img2(title, img1, img2):
    upper = len(img1) // 3
    lower = 2 * upper
    middle = len(img1) // 2
    result_middle = []
    k = 10
    for i in range(-k, k + 1):
        mse = mean_squared_error(img1[middle], img2[middle + i])
        result_middle.append([mse])
    index = result_middle.index(min(result_middle))
    return index - k

In [16]:
def merge_image(img_b, img_g, img_r, r_pass, g_pass, sort):
    b_second = img_b.shape[0] // 2
    r_second = img_r.shape[0] // 2 + r_pass
    g_second = img_g.shape[0] // 2 + g_pass
    min_second = min([b_second, g_second, r_second])
    b_first = img_b.shape[0] // 2
    g_first = img_g.shape[0] // 2 - g_pass
    r_first = img_b.shape[0] // 2 - r_pass
    min_first = min([b_first, r_first, g_first])
    img_b1 = img_b[img_b.shape[0] // 2 - min_first:img_b.shape[0] // 2 + min_second]
    img_g1 = img_g[img_g.shape[0] // 2 - min_first:img_g.shape[0] // 2 + min_second]
    img_r1 = img_r[img_r.shape[0] // 2 - min_first:img_r.shape[0] // 2 + min_second]
    if sort[0] == 0 and sort[1] == 1:
        return cv.merge([img_b1, img_r1, img_g1])
    elif sort[0] == 0 and sort[1] == 2:
        return cv.merge([img_b1, img_r1, img_g1])
    elif sort[0] == 1 and sort[1] == 2:
        return cv.merge([img_g1, img_r1, img_b1])
    elif sort[0] == 1 and sort[1] == 0:
        return cv.merge([img_g1, img_b1, img_r1])
    elif sort[0] == 2 and sort[1] == 0:
        return cv.merge([img_g1, img_r1, img_b1])
    elif sort[0] == 2 and sort[1] == 1:
        return cv.merge([img_r1, img_g1, img_b1])
    else:
        print("ok")


In [17]:
def img_preprocess(img_r,img_b,img_g):
    filters = build_filters()
    gabor_r = process(img_r, filters)
    gabor_g = process(img_g, filters)
    gabor_b = process(img_b, filters)
    imgs = [img_b, img_g, img_r]
    imgs2 = imgs.copy()
    imgs.sort(key=lambda x: x.shape[0])
    sort = [imgs2.index(element) for element in imgs]
    return gabor_b, gabor_g, gabor_r, imgs[0], imgs[1], imgs[2], sort

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
os.chdir("result")

In [20]:
merge_img=[]
for j in range(29):
    gabor_b, gabor_g, gabor_r, img_b, img_g, img_r, sort = img_preprocess(R[j],B[j],G[j])
    r_pass = equalization_img2("b-r", gabor_b, gabor_r)
    g_pass = equalization_img2("b-r", gabor_b, gabor_g)
    img = merge_image(img_b, img_g, img_r, r_pass, g_pass, sort)
    merge_img.append(img)
    cv.imwrite( str(j) + ".jpg", img)


In [ ]:
os.chdir("..")

In [32]:
img=merge_img[0]
cv.imshow("img",img)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
os.chdir("result_sharp")

In [29]:
sharp_img=[]
for i in merge_img:
    kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
    image_sharp = cv.filter2D(src=i, ddepth=-1, kernel=kernel)
    sharp_img.append(image_sharp)

In [30]:
save_img(sharp_img,"sharp")

Images saved for sharp space


In [ ]:
os.chdir("..")